## Sprint 3: Curate a dataset and simple models

### Step 1: Add fraud column

In [1]:
import pandas as pd
df = pd.read_csv('../../project-2-group-buy-now-pay-later-industry-project-4/data/tables/df_new_out.csv')
tmp1 = df['fraud_probability_x']>50 
tmp2 = df['fraud_probability_y']>50
df['is_fraud'] = tmp1 | tmp2
df.to_csv('../../project-2-group-buy-now-pay-later-industry-project-4/data/tables/df_new_week8.csv')

In [2]:
df.head(2)

,Unnamed: 0.1,Unnamed: 0,order_datetime,user_id,merchant_abn,dollar_value,fraud_probability_x,fraud_probability_y,consumer_id,name,address,state,postcode,gender,year,month,cpi,is_fraud
0,489,489,2021-09-02,260,90918180829,6708.812285,43.325517,11.181975,74257,Donald Alvarado,2982 Vickie Cliff,NSW,2869,Male,2021,9,107.3,False
1,490,490,2021-09-02,9653,90918180829,3542.290166,43.325517,9.146543,837975,Robert Parsons,4873 Hutchinson Stream,SA,5013,Male,2021,9,107.3,False


### A delta file of transactions to be treated as fraud is available. It is up to your group to remove these transactions or add a new field denoting fraud i.e an is fraud boolean field.

Our group decide to add fraud column according to merchant fraud probability or consumer fraud probability is greater than 0.5

### step2: Build a simple logistic model to predict whether fraud

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
df1 = df.dropna()
X = df1[['dollar_value','year','month','cpi']]
y = df1['is_fraud']
ohe = OneHotEncoder()
X_transformed = ohe.fit_transform(df1[['state','gender']]).toarray()
#print(ohe.categories_)
X_transformed = pd.DataFrame(X_transformed, columns=['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA','Female', 'Male', 'Undisclosed'])
for name in ['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA','Female', 'Male', 'Undisclosed']:
    X[name] = list(X_transformed[name])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1,stratify=y)
lr = LogisticRegression(C=100.0,random_state=1)
lr.fit(X_train,y_train)
print("Coef:",lr.coef_)
print("intercept",lr.intercept_)


Coef: [[-3.51350604e-05 -6.36274345e-02 -3.21376804e-02  1.15219159e+00
  -1.24943511e-02 -3.17147649e-02 -9.44915870e-02 -5.21654568e-02
   3.60436271e-01 -3.05849047e-01 -1.52053102e-01  2.88264556e-01
   2.87342412e-01 -1.18862243e-01 -1.68547651e-01]]
intercept [-6.80054138e-05]


/tmp/ipykernel_8301/3854025938.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[name] = list(X_transformed[name])
/tmp/ipykernel_8301/3854025938.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[name] = list(X_transformed[name])
/tmp/ipykernel_8301/3854025938.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

# result of logistic regression 

Positive coefficients indicate that the model is more likely to predict fraud when the value of the feature increases 

Negative coefficients indicate that the model is more likely to predict non-fraud when the value of the feature increases

In [4]:
from sklearn.metrics import confusion_matrix
y_pred = lr.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
print("confusion matrix ：")
print(conf_matrix)

confusion matrix ：
[[1355    0]
 [  10    0]]
